In this module, we'll explore Named Entity Recognition (NER) and Sentiment Analysis using Spark NLP with the Amazon reviews dataset. We'll identify the most common named entities mentioned in the reviews and analyze the overall sentiment of those reviews.

Import the necessary modules from PySpark and Spark NLP. These include functions and types from PySpark, and various annotators and base classes from Spark NLP.

In [0]:
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *

In [0]:
text_data_path = "dbfs:/databricks-datasets/amazon/data20K"
text_df = spark.read.parquet(text_data_path, header=True, inferSchema=True)

### Named Entity Recognition

Once our data is loaded, we're ready to start processing it. For our first task, we're going to identify named entities in the reviews. Named entities are real-world objects such as persons, locations, organizations, and so on, that can be denoted with a proper name.

Named Entity Recognition (NER) is a subtask of information extraction that seeks to locate and classify named entities in text into predefined categories such as person names, organizations, locations, medical codes, time expressions, quantities, monetary values, percentages, etc.

We'll create a processing pipeline with Spark NLP to do this. A pipeline is a sequence of stages where each stage is either a Transformer or an Estimator. These stages are run in order, and the input DataFrame is transformed as it passes through each stage.

In [0]:
# NER pipeline to identify and classify named entities in the reviews:

document_assembler = DocumentAssembler() \
    .setInputCol("review") \
    .setOutputCol("document")

sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentences")

tokenizer = Tokenizer() \
    .setInputCols(["sentences"]) \
    .setOutputCol("tokens")

embeddings = WordEmbeddingsModel.pretrained("glove_100d") \
    .setInputCols(["document", "tokens"]) \
    .setOutputCol("embeddings")

ner_model = NerDLModel.pretrained("ner_dl", "en") \
    .setInputCols(["sentences", "tokens", "embeddings"]) \
    .setOutputCol("ner_tags")

ner_converter = NerConverter() \
    .setInputCols(["sentences", "tokens", "ner_tags"]) \
    .setOutputCol("entities")

pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    embeddings,
    ner_model,
    ner_converter
])

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
ner_dl download started this may take some time.
Approximate size to download 13.6 MB
[OK!]


We're using the GloVe embeddings pretrained on 6 billion words ("glove_100d") to generate **word embeddings** which are then provided as input during named entity recognition. You can replace "glove_100d" with [any other embeddings](https://nlp.johnsnowlabs.com/models?type=model&q=glove) you prefer.

<img src="https://www.tensorflow.org/static/text/guide/images/embedding.jpg"  width="600" height="300">

The **NerDLModel** is a Named Entity Recognition model trained by a deep learning approach. It assigns to every word in a text a tag that signifies whether the word is a named entity or not, and what category it belongs to. It's trained using a variety of neural network architectures, such as Char CNNs - BiLSTM - CRF and Bert - BiLSTM - CRF.  
In the code above, we are loading a pretrained NER model (named "ner_dl") for English ("en"). We set the input columns to be "sentences" and "tokens". This means that the model will perform NER on the tokenized sentences from our text. The output of this stage is a new column, "ner_tags", which contains the NER tags assigned by the model to each word.

The **NerConverter** is used to convert the output from the NerDLModel into a more readable format. It groups together consecutive tokens with the same NER tag into single entities and classifies them according to their tag.  
We're setting the input columns to be "sentences", "tokens", and "ner_tags". This means that the NerConverter will take the tokenized sentences and their corresponding NER tags as input. The output of this stage is a new column, "entities", which contains the named entities extracted from the text. Each entity is represented as a chunk of text along with its NER tag.

We will now transform the reviews DataFrame using the NER pipeline:

In [0]:
pipeline_model = pipeline.fit(text_df)
ner_result_df = pipeline_model.transform(text_df)

In [0]:
ner_result_df.show(4)

# ner_result_df.select("review", "entities").show(2, truncate=False)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|rating|              review|            document|           sentences|              tokens|          embeddings|            ner_tags|            entities|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   4.0|Worked as expecte...|[{document, 0, 10...|[{document, 0, 62...|[{token, 0, 5, Wo...|[{word_embeddings...|[{named_entity, 0...|                  []|
|   5.0|This mouse is ama...|[{document, 0, 46...|[{document, 0, 69...|[{token, 0, 3, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 37, 46, ...|
|   4.0|we recently had a...|[{document, 0, 79...|[{document, 0, 94...|[{token, 0, 1, we...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 471, 472...|
|   3.0|Works good for a ...|[{document, 0, 23...|[{document, 0,

Spark NLP also offers pretrained pipeline. They are a pre-built set of transformations that have been trained on a large dataset and saved, so they can be reused. This is a great way to get started quickly with NLP tasks like Named Entity Recognition (NER), sentiment analysis, and more.

To use a pretrained pipeline, you don't need to assemble each individual component. Instead, you can load the entire pipeline at once. One catch is that pretrained pipelines assume that the input column is named "text".

Here's an example of how you can do this:

In [0]:
# from sparknlp.pretrained import PretrainedPipeline

# Load a pretrained pipeline
# pipeline = PretrainedPipeline("recognize_entities_dl", lang="en")

# Rename the target column - "review" -> "text"
# text_df_2 = test_df.withColumnRenamed("review", "text")

# Use the pipeline to annotate a DataFrame
# result_df = pipeline.transform(text_df_2)

- We import the PretrainedPipeline class from the sparknlp.pretrained module.
- We create an instance of the PretrainedPipeline class by calling the constructor and passing the name of the pretrained pipeline we want to use ("recognize_entities_dl") and the language ("en").
- We rename the input column from "review" to "text".
- We use the annotate method of the PretrainedPipeline instance to transform our DataFrame. The annotate method takes two arguments: the DataFrame to transform, and the name of the column in the DataFrame that contains the text to process.

In [0]:
named_entities = ner_result_df.select("entities.result").withColumnRenamed("result", "named_entities")

named_entities.show(5, truncate=False)

+------------------------+
|named_entities          |
+------------------------+
|[]                      |
|[Razer Naga, R.A.T, IRL]|
|[TV, Difference]        |
|[Said, Catan]           |
|[zipper]                |
+------------------------+
only showing top 5 rows



### Sentiment analysis


Next, we build another pipeline, this time for sentiment analysis. This pipeline is similar to our NER pipeline, but instead of the NER model, we use a pretrained sentiment analysis model.

In [0]:
document_assembler = DocumentAssembler() \
    .setInputCol("review") \
    .setOutputCol("document")

sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentences")

tokenizer = Tokenizer() \
    .setInputCols(["sentences"]) \
    .setOutputCol("tokens")

embeddings = UniversalSentenceEncoder.pretrained('tfhub_use', lang='en') \
    .setInputCols(["document"]) \
    .setOutputCol("sentence_embeddings")

sentiment_model = SentimentDLModel.pretrained("sentimentdl_use_imdb", "en") \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("sentiment")

pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    embeddings,
    sentiment_model
])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_imdb download started this may take some time.
Approximate size to download 12 MB
[OK!]


This pipeline first assembles the text into a document, then breaks the document into sentences and tokenizes those sentences.  
The pre-trained BertSentenceEmbeddings model then creates embeddings for each sentence, and finally, the SentimentDLModel uses these embeddings to predict sentiment.

EXERCISE: Why did we use sentence embeddings instead of word embeddings like we had done in the NER task?

We can now fit and transform our filtered reviews DataFrame using this new pipeline.

In [0]:
pipeline_model = pipeline.fit(text_df)
sentiment_result_df = pipeline_model.transform(text_df)

In [0]:
sentiment_result_df.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|rating|              review|            document|           sentences|              tokens| sentence_embeddings|           sentiment|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   4.0|Worked as expecte...|[{document, 0, 10...|[{document, 0, 62...|[{token, 0, 5, Wo...|[{sentence_embedd...|[{category, 0, 10...|
|   5.0|This mouse is ama...|[{document, 0, 46...|[{document, 0, 69...|[{token, 0, 3, Th...|[{sentence_embedd...|[{category, 0, 46...|
|   4.0|we recently had a...|[{document, 0, 79...|[{document, 0, 94...|[{token, 0, 1, we...|[{sentence_embedd...|[{category, 0, 79...|
|   3.0|Works good for a ...|[{document, 0, 23...|[{document, 0, 12...|[{token, 0, 4, Wo...|[{sentence_embedd...|[{category, 0, 23...|
|   2.0|Fabric is nice an...|[{document, 0, 14...|[{doc

In [0]:
sentiment_result_df.select("review", "sentiment.result").show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|review                                                                                                                                                                                             